In [1]:
import pandas as pd
import numpy as np
import torch 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torch_geometric.data import Data, Dataset
import dgl

In [2]:
import torch.nn.functional as F 
from torch.nn import Linear, BatchNorm1d, ModuleList
from torch_geometric.nn import TransformerConv, TopKPooling, GATConv, SAGEConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [4]:
device 

device(type='cuda', index=3)

In [5]:
x = torch.tensor([1,0,1,1,0,1,1])
print(x.shape)
print(F.one_hot(x))

torch.Size([7])
tensor([[0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1]])


# Need to define the data class 
- Here focus mainly on the get() method. We don't need to process anything
- We also return masks for each graph, that will help with training 

In [6]:
class WSI_Graph_Class(Dataset):
    
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(None, transform, pre_transform)
        self.root_dir = root
        self.WSI_df = pd.read_csv(root) #get the WSI metadata
        self.masks = {} #map node num -> train/val/test masks
        self.create_all_masks()
        
    def create_all_masks(self):
        
        for idx in tqdm(range(len(self.WSI_df["sample_id"]))):
            path = self.WSI_df["path"].iloc[idx]
            
            #this is the graph. We also need to return the training/validation/testing masks 
            data = torch.load(path)
            nodes = [i for i in range(data.x.shape[0])] #node 0 is in 0th pos, 1 in 1, and so on 
            
            #all of the masks 
            train_mask = [False] * len(nodes)
            val_mask = [False] * len(nodes)
            test_mask = [False] * len(nodes)
            self.create_mask(nodes, train_mask, val_mask, test_mask)
            #now add them to dictionary
            self.masks[idx] = [train_mask, val_mask, test_mask]
        
        
    def create_mask(self, nodes, train_mask, val_mask, test_mask):        
        #create train/test/val nodes (75/25)
        train, test = train_test_split(nodes)
        test, val = train_test_split(test)
        
        #now create masks
        for i in range(len(nodes)):
            if i in train: 
                train_mask[i] = True 
                
        for i in range(len(nodes)):
            if nodes[i] in val: 
                val_mask[i] = True 
                
        for i in range(len(nodes)):
            if nodes[i] in test: 
                test_mask[i] = True 
                
    #just pass here, we aren't going to return any raw file names
    def raw_file_names(self):
        pass 
    #here we can return each of the WSI 
    def processed_file_names(self):
        return list(self.WSI_df["sample_id"])
    
    def len(self):
        return len(self.processed_file_names())
    
    #return the graph class for that idx 
    def get(self, idx):
        path = self.WSI_df["path"].iloc[idx]
        #this is the graph. We also need to return the training/validation/testing masks 
        data = torch.load(path)
        masks = self.masks[idx]
        train_mask = masks[0]
        val_mask = masks[1]
        test_mask = masks[2]
        return (data, torch.tensor(train_mask), torch.tensor(val_mask), torch.tensor(test_mask))

In [7]:
root = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/graph_data/metadata.csv"

dataset = WSI_Graph_Class(root = root, transform = None, pre_transform = None, pre_filter = None)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:03<00:00,  2.10s/it]


In [8]:
data = dataset.get(1)

# Data Sanity Check 

In [9]:
data

(Data(x=[14807, 2048], edge_index=[2, 113718], y=[14807]),
 tensor([False,  True, False,  ...,  True,  True,  True]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([ True, False,  True,  ..., False, False, False]))

In [10]:
sum(data[0].y == 1)/sum(data[0].y == 0)

tensor(0.3422)

In [11]:
graph = data[0]

In [12]:
graph.x

tensor([[0.3273, 0.3910, 0.2434,  ..., 0.1320, 0.3427, 0.1643],
        [0.3633, 0.4324, 0.2210,  ..., 0.1202, 0.3847, 0.1370],
        [0.3143, 0.3583, 0.1967,  ..., 0.1235, 0.3223, 0.1408],
        ...,
        [0.3529, 0.3735, 0.1997,  ..., 0.1163, 0.3923, 0.1283],
        [0.3563, 0.4631, 0.2405,  ..., 0.1372, 0.4183, 0.1589],
        [0.3672, 0.4138, 0.1967,  ..., 0.1255, 0.4014, 0.1501]])

In [13]:
sum(graph.y == 1)

tensor(3775)

In [14]:
val_mask = data[2]
train_mask = data[1]

In [15]:
sum(graph.y[train_mask] == 1)

tensor(2825)

In [16]:
sum(graph.y[train_mask] == 0)

tensor(8280)

In [17]:
list(graph.x[0])

[tensor(0.3273),
 tensor(0.3910),
 tensor(0.2434),
 tensor(0.2740),
 tensor(0.3770),
 tensor(0.1986),
 tensor(0.2894),
 tensor(0.2798),
 tensor(0.2180),
 tensor(0.2774),
 tensor(0.2228),
 tensor(0.2582),
 tensor(0.2934),
 tensor(0.2616),
 tensor(0.2318),
 tensor(0.2059),
 tensor(0.2734),
 tensor(0.1345),
 tensor(0.1636),
 tensor(0.2380),
 tensor(0.2302),
 tensor(0.1834),
 tensor(0.3113),
 tensor(0.2886),
 tensor(0.2514),
 tensor(0.3097),
 tensor(0.2254),
 tensor(0.2886),
 tensor(0.3816),
 tensor(0.1610),
 tensor(0.1412),
 tensor(0.1973),
 tensor(0.1213),
 tensor(0.2101),
 tensor(0.3877),
 tensor(0.1789),
 tensor(0.3065),
 tensor(0.2127),
 tensor(0.2804),
 tensor(0.2757),
 tensor(0.2536),
 tensor(0.4117),
 tensor(0.2085),
 tensor(0.2539),
 tensor(0.3148),
 tensor(0.3571),
 tensor(0.1836),
 tensor(0.3520),
 tensor(0.1677),
 tensor(0.1936),
 tensor(0.2729),
 tensor(0.1984),
 tensor(0.3282),
 tensor(0.2191),
 tensor(0.3683),
 tensor(0.3351),
 tensor(0.2762),
 tensor(0.1524),
 tensor(0.4025

In [18]:
#how similar are the patch embeddings 
sum((graph.x[0] - graph.x[8])**2) #so these embeddings are different

tensor(1.8893)

# Make the train set just embeddings and output 

In [38]:
from torch_geometric.data import DataLoader
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [39]:
xs = []
ys = []

In [40]:
for data in tqdm(data_loader):
    x = data[0].x
    y = data[0].y
    for t in x:
        xs.append(t)
    for t in y:
        ys.append(t)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:05<00:00,  5.37it/s]


In [41]:
train_x, test_x, train_y, test_y = train_test_split(xs, ys, test_size = .25)

In [42]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = .25)

In [43]:
print(len(train_x), len(val_x), len(test_x))

246662 82221 109628


# Define Model 
- This mainly draws upon HIV project code 

In [53]:
class simple_NN(torch.nn.Module):
    def __init__(self):
        super(simple_NN, self).__init__()
        

        self.linear = Linear(2048, 2048) 
        self.linear2 = Linear(2048, 100)
        self.linear3 = Linear(100, 2)

    def forward(self, x):
        
        x = F.relu(self.linear(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x

# Dataloader

In [54]:
num_features = 2048
model = simple_NN()
model = model.to(device)

In [55]:
#loss and optimizer 
import torch.optim as optim
import torchvision.ops.focal_loss

# loss_fn = torchvision.ops.focal_loss.sigmoid_focal_loss
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=5e-4)

In [56]:
#prepare training 
train_loader = DataLoader(list(zip(train_x, train_y)), batch_size=64, shuffle=True)
val_loader = DataLoader(list(zip(val_x, val_y)), batch_size=64, shuffle=True)
num_epochs = 10

In [57]:
softmax = nn.Softmax(dim=1)

# Actual Model Training 

In [59]:
for epoch in range(num_epochs):
    #training portion
    epoch_loss = []
    model.train()
    for data in tqdm(train_loader):
        #get graph
        x = data[0]
        y = data[1]
        #move to device
        x = x.to(device)
        y = y.to(device)

        #get predictions 
        logits = model(x)
        loss = loss_fn(logits, y) #for CE

        epoch_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()
        opt.step()
    #now find the average training loss for this epoch 
    epoch_loss = sum(epoch_loss)/len(epoch_loss)
    print("Epoch :%d. Epoch loss: %f" %(epoch, epoch_loss))    
    #validation portion
    validation_correct = 0
    validation_total = 0
    model.eval()
    with torch.no_grad():
        for data in tqdm(val_loader):
            #get graph
            x = data[0]
            y = data[1]
            #move to device
            x = x.to(device)
            y = y.to(device)
            
            #get predictions 
            logits = model(x)
            #get them into label predictions
            _, indices = torch.max(logits, dim=1)
#             print(indices)
            validation_correct += sum(indices == y).item()
            validation_total += len(y)
#             print("Accuracy on this graph's val set", sum(indices == y).item()/len(y))
#             print("SCC percent", sum(y == 1).item()/len(y))
    
    print("Epoch :%d. Validation accuracy: %f" %(epoch, validation_correct/validation_total))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3855/3855 [00:07<00:00, 500.27it/s]


Epoch :0. Epoch loss: 0.641772


  2%|███▍                                                                                                                                                                | 27/1285 [00:00<00:04, 264.92it/s]

Accuracy on this graph's val set 0.578125
SCC percent 0.515625
Accuracy on this graph's val set 0.609375
SCC percent 0.4375
Accuracy on this graph's val set 0.625
SCC percent 0.5
Accuracy on this graph's val set 0.609375
SCC percent 0.53125
Accuracy on this graph's val set 0.65625
SCC percent 0.515625
Accuracy on this graph's val set 0.640625
SCC percent 0.484375
Accuracy on this graph's val set 0.609375
SCC percent 0.421875
Accuracy on this graph's val set 0.625
SCC percent 0.5625
Accuracy on this graph's val set 0.703125
SCC percent 0.453125
Accuracy on this graph's val set 0.640625
SCC percent 0.515625
Accuracy on this graph's val set 0.640625
SCC percent 0.546875
Accuracy on this graph's val set 0.640625
SCC percent 0.515625
Accuracy on this graph's val set 0.609375
SCC percent 0.546875
Accuracy on this graph's val set 0.59375
SCC percent 0.53125
Accuracy on this graph's val set 0.6875
SCC percent 0.359375
Accuracy on this graph's val set 0.578125
SCC percent 0.40625
Accuracy on th

  5%|███████▌                                                                                                                                                            | 59/1285 [00:00<00:04, 296.57it/s]

...

 63%|█████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                             | 804/1285 [00:02<00:01, 410.13it/s]

Accuracy on this graph's val set 0.6875
SCC percent 0.484375
Accuracy on this graph's val set 0.5625
SCC percent 0.484375
Accuracy on this graph's val set 0.671875
SCC percent 0.46875
Accuracy on this graph's val set 0.671875
SCC percent 0.46875
Accuracy on this graph's val set 0.59375
SCC percent 0.5625
Accuracy on this graph's val set 0.6875
SCC percent 0.484375
Accuracy on this graph's val set 0.703125
SCC percent 0.484375
Accuracy on this graph's val set 0.625
SCC percent 0.546875
Accuracy on this graph's val set 0.59375
SCC percent 0.5
Accuracy on this graph's val set 0.671875
SCC percent 0.46875
Accuracy on this graph's val set 0.640625
SCC percent 0.53125
Accuracy on this graph's val set 0.59375
SCC percent 0.5625
Accuracy on this graph's val set 0.46875
SCC percent 0.515625
Accuracy on this graph's val set 0.5625
SCC percent 0.546875
Accuracy on this graph's val set 0.59375
SCC percent 0.53125
Accuracy on this graph's val set 0.65625
SCC percent 0.484375
Accuracy on this graph'

 70%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                | 905/1285 [00:02<00:00, 380.18it/s]

Accuracy on this graph's val set 0.640625
SCC percent 0.5625
Accuracy on this graph's val set 0.59375
SCC percent 0.53125
Accuracy on this graph's val set 0.484375
SCC percent 0.515625
Accuracy on this graph's val set 0.609375
SCC percent 0.546875
Accuracy on this graph's val set 0.640625
SCC percent 0.5625
Accuracy on this graph's val set 0.640625
SCC percent 0.484375
Accuracy on this graph's val set 0.703125
SCC percent 0.4375
Accuracy on this graph's val set 0.59375
SCC percent 0.5
Accuracy on this graph's val set 0.671875
SCC percent 0.421875
Accuracy on this graph's val set 0.703125
SCC percent 0.453125
Accuracy on this graph's val set 0.6875
SCC percent 0.375
Accuracy on this graph's val set 0.75
SCC percent 0.453125
Accuracy on this graph's val set 0.5625
SCC percent 0.53125
Accuracy on this graph's val set 0.671875
SCC percent 0.53125
Accuracy on this graph's val set 0.546875
SCC percent 0.484375
Accuracy on this graph's val set 0.671875
SCC percent 0.515625
Accuracy on this gr

KeyboardInterrupt: 

In [ ]:
 #test portion
test_correct = 0
test_total = 0
model.eval()
with torch.no_grad():
    for data in tqdm(data_loader):
        #get graph
        graph = data[0]
        x = graph.x 
        edge_index = graph.edge_index
        y = graph.y 
        #move to device
        x = x.to(device)
        edge_index = edge_index.to(device)
        y = y.to(device)
        #get masks
        test_mask = data[3].T.reshape([data[3].T.shape[0]])

        #get predictions 
        logits = model(x, edge_index)
        #get them into label predictions
        _, indices = torch.max(logits, dim=1)
        print(1 in indices)
        test_correct += sum(indices[test_mask] == y[test_mask]).item()
        test_total += sum(test_mask == True).item()

print("Test accuracy: %f" %(test_correct/test_total))

In [16]:
!nvidia-smi

Sat Dec 24 02:12:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   37C    P0    54W / 300W |   3046MiB / 32768MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [17]:
torch.cuda.device_count()

4